In [1]:
import os
from dotenv import load_dotenv
import openai
from langchain_community.vectorstores import FAISS, Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from crewai import Agent, Task, Crew, Process
from crewai_tools import BaseTool
from flask import Flask, request, jsonify
import os
os.environ["SERPER_API_KEY"] = "7e5445663d8bf0f45167042f32eaeea46426395d"  # serper.dev API key

from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

# Load environment variables
load_dotenv()

# Set Azure OpenAI environment variables
os.environ["AZURE_OPENAI_KEY"] = "14a38573c7c04028af5e11f1e158ec71"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openaihumanvaluex.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt4o"
os.environ["AZURE_OPENAI_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_EMBEDDINGS_MODEL_NAME"] = "adaembedding"
os.environ['OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_KEY")

# Configure OpenAI API key and version
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
api_version = os.getenv("AZURE_OPENAI_VERSION")

# Initialize Azure services
llm = AzureChatOpenAI(
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    openai_api_version=api_version
)
embedding = AzureOpenAIEmbeddings(
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME"),
    openai_api_version=api_version,
)

vectorstore = Chroma(persist_directory="SaudiAramcoFINSddA", embedding_function=embedding, collection_name="Findata_SaudiAramco_2021")
retriever = vectorstore.as_retriever()

class ListFinancialDataTool(BaseTool):
    name: str = "Financial Data table list tool"
    description: str = "Lists the set of the core financial data table names available within the Annual Report"

    def _run(self, query: str) -> str:
        # Implementation goes here
        table_names = [
             'Consolidated Statement of Income',
             'Consolidated Statement of Cash Flows',
             'Consolidated Balance Sheet' 
         ]
        return table_names

# Define custom tool class for yoga practices search
class FinancialDataTool(BaseTool):
    name: str = "Financial Data table extraction tool"
    description: str = "Extracts the financial data table based on the name of the table"

    def _run(self, query: str) -> str:
        # Implementation goes here
        return retriever.get_relevant_documents(query)

# Create agent and assign tools
tools = [ListFinancialDataTool(),FinancialDataTool()]
Income_statement_analyst_agent = Agent(
    role='Senior Financial Analyst',
    goal='Identify revenue trends, expense patterns, and profit margins to uncover insights for cross-sell and up-sell opportunities for the customer',
    backstory="you are a senior relationship Manager at a international Bank with 20 years of experience in relation ship Management and sales and is looking for cross sell and upsell opportunity with wholesale bankign customers",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools=tools
)
Balance_sheet_analyst_agent = Agent(
    role='Senior Financial Analyst',
    goal='Assess asset composition, liability structure, and equity levels to identify financial needs for wholesale banking clients.',
    backstory="you are a senior relationship Manager and financial analyst at a international Bank with 20 years of experience in relation ship Management and sales and is looking for cross sell and upsell opportunity with wholesale banking customers",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools=tools
)

Cash_flow_analyst_agent = Agent(
    role='Senior Financial Analyst',
    goal='Understand cash flow dynamics to identify liquidity needs and investment opportunities for wholesale banking clients.',
    backstory="you are a senior relationship Manager and financial analyst at a international Bank with 20 years of experience in relation ship Management and sales and is looking for cross sell and upsell opportunity with wholesale banking customers",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools=tools
)


Senior_Wholesale_Banking_Product_Manager = Agent(
  role='Senior Wholesale Banking Product Manager',
  goal='Analyze insights and highlights from financial analysts to develop and enhance banking products and provide strategic investment recommendations to institutional clients.',
  backstory="""You are a Senior Wholesale Banking Product Manager with a strong background in finance and extensive experience in the banking industry. You are adept at interpreting complex financial data and transforming it into innovative product propositions and strategic investment advice. Your goal is to leverage financial analysis to create competitive banking products and provide high-value investment recommendations that align with the strategic objectives of institutional clients.""",
  llm=llm,
  verbose=True,
  allow_delegation=False
)



# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Financial Researcher',
  goal='Uncover the news and announcment related to the Saudi Aramco to provide insights to the wholesale banking Product Manager to make informed decisions',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "information linked to Saudi Aramco, eager to share the information that would help the wholesale banking product manager to customize its product proposition and personalise the recommendations"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True,
  llm=llm
)




task1 = Task(
    description="""Perform the financial analysis based on the Annual report for Saudi Aramco using the income statements data. 
    Some of the instructions are:
    - Analyze revenue streams and growth trends.
    - Examine major expense categories.
    - Calculate profit margins and evaluate profitability.
    Some of the rules of conditions include:
    - Revenue growth > 10% YoY: Cross-sell working capital solutions, trade finance, and foreign exchange services.
    - Operating expenses > 30% of revenue: Up-sell cost management products like treasury management and cash flow optimization.
    - Profit margins > 15%: Cross-sell investment banking services such as M&A advisory; < 5%: Up-sell financing solutions like debt restructuring.
    
    """,
    expected_output="Insights into potential financial product needs based on revenue and expense analysis.",
    agent=Income_statement_analyst_agent,
    tools=tools,

)

task2 = Task(
  description="""Perform the financial analysis based on the Annual report for Saudi Aramco using the Balance Sheet statement data. 
    Some of the instructions are:
    - Review current and non-current assets.
    - Analyze short-term and long-term liabilities.
    - Evaluate equity-to-debt ratio.
    Some of the rules of conditions include:
    - High receivables: Cross-sell receivable financing or factoring services.
    - High short-term liabilities: Cross-sell working capital management or supply chain financing.
    - High long-term debt: Up-sell refinancing options, structured finance, or syndicated loans.
    - Low leverage: Cross-sell credit facilities; High leverage: Up-sell risk management solutions like interest rate hedging.
    """,
    
  expected_output="Detailed analysis of asset, liability, and equity positions, highlighting potential opportunities.",
  agent=Balance_sheet_analyst_agent,
  tools=tools,
  context=[task1],
 
)

task3 = Task(
  description="""Perform the financial analysis based on the Annual report for Saudi Aramco using the Cash flow data. 
    Some of the instructions are:
    - Evaluate operating, investing, and financing cash flows.
    - Identify patterns in cash inflows and outflows.
    Some of the rules of conditions include:
    - Positive operating cash flow: Cross-sell investment products, cash management services, or liquidity management solutions.
    - Negative operating cash flow: Up-sell liquidity solutions like revolving credit lines or bridge loans.
    - High capital expenditures: Cross-sell project financing, equipment leasing, or infrastructure finance.
    - Equity financing activities: Up-sell advisory services for capital raising, IPOs, or private placements.
    """,
    
  expected_output="Cash flow insights indicating liquidity needs and potential investment or advisory services.",
  agent=Senior_Wholesale_Banking_Product_Manager,
  tools=tools,
  context=[task1, task2]
)

task5 = Task(
  description="""Review the latest news and announcements related to Saudi Aramco to provide insights to the wholesale banking Product Manager to make informed decisions.""",
  expected_output="Key announcement and insights identifying either risks or opportunities for the bank.",
  agent=researcher,
  tools=[search_tool],

)


task4 = Task(
  description="""Based on the insights and highlights from the financial analysts, develop and enhance banking products and provide strategic investment recommendations to institutional clients.
    Existing products include:
    - Working capital solutions for revenue growth with credit facility of up to $10 million.
    - Account services with net balance of $5 million.
    Some of the rules of conditions include:
    - RAROC should be > 10% for new products.
    
    """,
  expected_output="Strategic investment recommendations and innovative banking product propositions.",
  agent=Cash_flow_analyst_agent,
  tools=tools,
  context=[task1, task2, task3, task5],
)




crew = Crew(
    agents=[Income_statement_analyst_agent,Balance_sheet_analyst_agent, Cash_flow_analyst_agent, Senior_Wholesale_Banking_Product_Manager],
    tasks=[task1, task2, task3, task5, task4],
    verbose=2

)

# Execute and print results
result = crew.kickoff()
print("######################")
print(result)

/Users/govind/.pyenv/versions/3.10.4/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(
2024-10-12 16:47:15,676 - 7894802112 - manager.py-manager:293 - WARNING: Error in TokenCalcHandler.on_llm_start callback: AttributeError("'NoneType' object has no attribute 'startswith'")


 [2024-10-12 16:47:15][DEBUG]: == Working Agent: Senior Financial Analyst
 [2024-10-12 16:47:15][INFO]: == Starting Task: Perform the financial analysis based on the Annual report for Saudi Aramco using the income statements data. 
    Some of the instructions are:
    - Analyze revenue streams and growth trends.
    - Examine major expense categories.
    - Calculate profit margins and evaluate profitability.
    Some of the rules of conditions include:
    - Revenue growth > 10% YoY: Cross-sell working capital solutions, trade finance, and foreign exchange services.
    - Operating expenses > 30% of revenue: Up-sell cost management products like treasury management and cash flow optimization.
    - Profit margins > 15%: Cross-sell investment banking services such as M&A advisory; < 5%: Up-sell financing solutions like debt restructuring.
    
    


> Entering new CrewAgentExecutor chain...


2024-10-12 16:47:19,248 - 7894802112 - manager.py-manager:293 - WARNING: Error in TokenCalcHandler.on_llm_start callback: AttributeError("'NoneType' object has no attribute 'startswith'")


To perform the financial analysis based on the Annual Report for Saudi Aramco, I need to first identify and extract the relevant financial data tables, particularly the income statements. 

Thought: The first step is to list the core financial data tables available within the Annual Report to identify the precise name of the income statement table.

Action: Financial Data table list tool
Action Input: {"query": "List the core financial data tables available within the Annual Report for Saudi Aramco"}
 

I encountered an error while trying to use the tool. This was the error: StructuredTool._run() missing 1 required keyword-only argument: 'config'.
 Tool Financial Data table list tool accepts these inputs: Financial Data table list tool(query: 'string') - Lists the set of the core financial data table names available within the Annual Report



2024-10-12 16:47:20,758 - 7894802112 - manager.py-manager:293 - WARNING: Error in TokenCalcHandler.on_llm_start callback: AttributeError("'NoneType' object has no attribute 'startswith'")


Thought: To proceed with the financial analysis, I need to identify the income statement table from the Annual Report for Saudi Aramco. Since the Financial Data table list tool is required to find the table name, I will attempt to use it again with the proper input.

Action: Financial Data table list tool
Action Input: {"query": "List the core financial data tables available within the Annual Report for Saudi Aramco"}
 

I encountered an error while trying to use the tool. This was the error: StructuredTool._run() missing 1 required keyword-only argument: 'config'.
 Tool Financial Data table list tool accepts these inputs: Financial Data table list tool(query: 'string') - Lists the set of the core financial data table names available within the Annual Report



2024-10-12 16:47:23,048 - 7894802112 - manager.py-manager:293 - WARNING: Error in TokenCalcHandler.on_llm_start callback: AttributeError("'NoneType' object has no attribute 'startswith'")


Thought: Since the Financial Data table list tool is not functioning correctly, I will delegate the task of identifying the income statement table name to a coworker. This will allow me to proceed with extracting the necessary financial data for analysis.

Action: Delegate work to coworker
Action Input: {"task": "Identify the name of the income statement table within the Annual Report for Saudi Aramco.", "context": "We need to perform financial analysis based on the Annual Report for Saudi Aramco. The first step is to identify the income statement table name so we can extract the relevant data for analysis.", "coworker": "Senior Financial Analyst"}
 

I encountered an error while trying to use the tool. This was the error: StructuredTool._run() missing 1 required keyword-only argument: 'config'.
 Tool Delegate work to coworker accepts these inputs: Delegate a specific task to one of the following coworkers: [Senior Financial Analyst, Senior Financial Analyst, Senior Wholesale Banking P

KeyboardInterrupt: 